In [1]:
%load_ext autoreload
%autoreload 2

## Create a simplified cord19 app

In [47]:
from vespa.package import Document, Field, Schema, FieldSet, RankProfile, ApplicationPackage

document = Document(
    fields=[
        Field(name = "cord_uid", type = "string", indexing = ["attribute", "summary"]),
        Field(name = "title", type = "string", indexing = ["index", "summary"], index = "enable-bm25"),
    ]
)

msmarco_schema = Schema(
    name = "cord19",
    document = document,
    fieldsets = [FieldSet(name = "default", fields = ["title"])],
    rank_profiles = [RankProfile(name = "default", first_phase = "nativeRank(title)")]
)

app_package = ApplicationPackage(name = "c", schema=msmarco_schema)

In [1]:
from vespa.package import VespaDocker

vespa_docker = VespaDocker()

In [49]:
vespa_docker.export_application_package(dir_path="cord19/sample_application", application_package=app_package)

Steps required to deploy BERT:

* Add onnx-model in the sd.
* Create a models folder in the same level of the schemas folder
* Create a rank-profile that define inputs and use the model to rank
* Add the input field tensors related to the docs
* Add query profile with the relevant tensors

## Deploy application from disk

In [34]:
vespa_docker.container = None

In [35]:
vespa_docker.container

In [9]:
app = vespa_docker.deploy_from_disk(
    application_name="cord19", 
    disk_folder="/Users/tmartins/projects/vespa/pyvespa/docs/sphinx/source/use_cases/cord19/sample_application", 
    container_memory="10G"
)

In [10]:
app.deployment_message

["Uploading application '/app/application' using http://localhost:19071/application/v2/tenant/default/session",
 "Session 4 for tenant 'default' created.",
 'Preparing session 4 using http://localhost:19071/application/v2/tenant/default/session/4/prepared',
 "WARNING: Host named 'cord19' may not receive any config since it is not a canonical hostname. Disregard this warning when testing in a Docker container.",
 "Session 4 for tenant 'default' prepared.",
 'Activating session 4 using http://localhost:19071/application/v2/tenant/default/session/4/active',
 "Session 4 for tenant 'default' activated.",
 'Checksum:   a0076d5553528905a59e92d1f310fa81',
 'Timestamp:  1603224854380',
 'Generation: 4',
 '']

## Feed data to the application

In [11]:
import json

test_sets = json.load(open("cord19/test_sets.json", "r"))

In [3]:
test_sets

{'partially_observed': [{'query_id': 1,
   'query': 'coronavirus origin',
   'relevant_docs': [{'id': '005b2j4b', 'score': 2},
    {'id': '010vptx3', 'score': 2},
    {'id': '0194oljo', 'score': 1},
    {'id': '0evw0fc5', 'score': 1},
    {'id': '0t2a5500', 'score': 2},
    {'id': '127c5bve', 'score': 2},
    {'id': '1dogron2', 'score': 1},
    {'id': '1mjaycee', 'score': 2},
    {'id': '1spgrrmw', 'score': 1},
    {'id': '23yi8so0', 'score': 2},
    {'id': '2b25p7t7', 'score': 2},
    {'id': '2ftw85xw', 'score': 2},
    {'id': '2ytec133', 'score': 1},
    {'id': '330hioeg', 'score': 1},
    {'id': '3jgedokv', 'score': 2},
    {'id': '3ta5lhnw', 'score': 1},
    {'id': '49360l2a', 'score': 2},
    {'id': '5283jsnu', 'score': 1},
    {'id': '55loucvc', 'score': 2},
    {'id': '59492sjb', 'score': 2},
    {'id': '5f42du0b', 'score': 2},
    {'id': '5fg87lvu', 'score': 2},
    {'id': '5h7qyn1g', 'score': 1},
    {'id': '5l10cbp2', 'score': 1},
    {'id': '69fmtury', 'score': 2},
    {'id'

In [25]:
documents_to_feed = []
for test_set in test_sets:
    for query_point in test_sets[test_set]:
        query = query_point["query"]
        print(query)
        result = app.query(
            query=query, 
            query_model=Query(
                match_phase = OR(),
                rank_profile = Ranking(name="bm25")
            ),
            timeout="20s",    
            hits = 100
        )
        assert len(result.hits) > 0
        for hit in result.hits:
            documents_to_feed.append(
                {"cord_uid": hit["fields"]["cord_uid"],
                 "title": hit["fields"]["title-full"]}
            )

coronavirus origin
coronavirus response to weather changes
how do people die from the coronavirus
animal models of COVID-19
coronavirus test rapid testing
serological tests for coronavirus
coronavirus under reporting
coronavirus in Canada
coronavirus hospital rationing
coronavirus quarantine
coronavirus super spreaders
how long does coronavirus survive on surfaces
coronavirus clinical trials
masks prevent coronavirus
what alcohol sanitizer kills coronavirus
coronavirus and ACE inhibitors
coronavirus mortality
coronavirus heart impacts
coronavirus hypertension
coronavirus diabetes
coronavirus biomarkers
coronavirus early symptoms
coronavirus hydroxychloroquine
coronavirus drug repurposing
coronavirus remdesivir
difference between coronavirus and flu
coronavirus subtypes
coronavirus vaccine candidates
coronavirus public datasets
SARS-CoV-2 spike structure
SARS-CoV-2 phylogenetic analysis
COVID inflammatory response
COVID-19 cytokine storm
coronavirus mutations
Vitamin D and COVID-19
viol

In [26]:
import json

with open("cord19/documents_to_feed.json", "w") as f:
    f.write(json.dumps(documents_to_feed))

In [14]:
import json

with open("cord19/documents_to_feed.json", "r") as f:
    documents_to_feed = json.load(f)

In [38]:
documents_to_feed[0:2]

[{'cord_uid': 'f047dvb4',
  'title': 'Retrouver les origines du SARS-CoV-2 dans les phylogénies de coronavirus./ [Tracing the origins of SARS-COV-2 in coronavirus phylogenies]'},
 {'cord_uid': 'bc8taaw9',
  'title': '[Tracing the origins of SARS-COV-2 in coronavirus phylogenies].'}]

In [131]:
training_data_batch.head()

,document_id,query_id,label,title-full,abstract-full
0,00fmeepz,1,1,Comprehensive overview of COVID-19 based on cu...,"In December 2019, twenty-seven pneumonia patie..."
1,f047dvb4,1,0,Retrouver les origines du SARS-CoV-2 dans les ...,"SARS-CoV-2 is a new human coronavirus (CoV), w..."
2,bc8taaw9,1,0,[Tracing the origins of SARS-COV-2 in coronavi...,"SARS-CoV-2 is a new human coronavirus (CoV), w..."
3,021q9884,1,1,Deadly virus effortlessly hops species,Genetic engineering helps reveal origin of dea...
4,f047dvb4,1,0,Retrouver les origines du SARS-CoV-2 dans les ...,"SARS-CoV-2 is a new human coronavirus (CoV), w..."


In [16]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [39]:
for document in documents_to_feed:
    response = app.feed_data_point(
        schema = "cord19",
        data_id = str(document["cord_uid"]),
        fields = {
            "cord_uid": str(document["cord_uid"]),
            "title": str(document["title"]),
            "title_token_ids": {"values": tokenizer(
                str(document["title"]), 
                truncation=True, 
                padding="max_length",
                max_length=63, 
                add_special_tokens=False
            )["input_ids"]}
        }
    )
    print(response.text)

{"id":"id:cord19:cord19::f047dvb4","pathId":"/document/v1/cord19/cord19/docid/f047dvb4"}
{"id":"id:cord19:cord19::bc8taaw9","pathId":"/document/v1/cord19/cord19/docid/bc8taaw9"}
{"id":"id:cord19:cord19::12dcftwt","pathId":"/document/v1/cord19/cord19/docid/12dcftwt"}
{"id":"id:cord19:cord19::kqqantwg","pathId":"/document/v1/cord19/cord19/docid/kqqantwg"}
{"id":"id:cord19:cord19::75773gwg","pathId":"/document/v1/cord19/cord19/docid/75773gwg"}
{"id":"id:cord19:cord19::hl967ekh","pathId":"/document/v1/cord19/cord19/docid/hl967ekh"}
{"id":"id:cord19:cord19::75773gwg","pathId":"/document/v1/cord19/cord19/docid/75773gwg"}
{"id":"id:cord19:cord19::4dtk1kyh","pathId":"/document/v1/cord19/cord19/docid/4dtk1kyh"}
{"id":"id:cord19:cord19::es7q6c90","pathId":"/document/v1/cord19/cord19/docid/es7q6c90"}
{"id":"id:cord19:cord19::0194oljo","pathId":"/document/v1/cord19/cord19/docid/0194oljo"}
{"id":"id:cord19:cord19::t1iagum7","pathId":"/document/v1/cord19/cord19/docid/t1iagum7"}
{"id":"id:cord19:cord

{"id":"id:cord19:cord19::xeu35i0n","pathId":"/document/v1/cord19/cord19/docid/xeu35i0n"}
{"id":"id:cord19:cord19::o3vm1vdh","pathId":"/document/v1/cord19/cord19/docid/o3vm1vdh"}
{"id":"id:cord19:cord19::kftchnhz","pathId":"/document/v1/cord19/cord19/docid/kftchnhz"}
{"id":"id:cord19:cord19::526elsrf","pathId":"/document/v1/cord19/cord19/docid/526elsrf"}
{"id":"id:cord19:cord19::joe9e4dh","pathId":"/document/v1/cord19/cord19/docid/joe9e4dh"}
{"id":"id:cord19:cord19::tjhdri57","pathId":"/document/v1/cord19/cord19/docid/tjhdri57"}
{"id":"id:cord19:cord19::cxtg9c2m","pathId":"/document/v1/cord19/cord19/docid/cxtg9c2m"}
{"id":"id:cord19:cord19::exqza1kg","pathId":"/document/v1/cord19/cord19/docid/exqza1kg"}
{"id":"id:cord19:cord19::lv8dvdp7","pathId":"/document/v1/cord19/cord19/docid/lv8dvdp7"}
{"id":"id:cord19:cord19::akhsxs1i","pathId":"/document/v1/cord19/cord19/docid/akhsxs1i"}
{"id":"id:cord19:cord19::r7fhr7qa","pathId":"/document/v1/cord19/cord19/docid/r7fhr7qa"}
{"id":"id:cord19:cord

{"id":"id:cord19:cord19::12pwvkf5","pathId":"/document/v1/cord19/cord19/docid/12pwvkf5"}
{"id":"id:cord19:cord19::7qatfxoj","pathId":"/document/v1/cord19/cord19/docid/7qatfxoj"}
{"id":"id:cord19:cord19::076zfaic","pathId":"/document/v1/cord19/cord19/docid/076zfaic"}
{"id":"id:cord19:cord19::hg02e3t6","pathId":"/document/v1/cord19/cord19/docid/hg02e3t6"}
{"id":"id:cord19:cord19::fowfz59x","pathId":"/document/v1/cord19/cord19/docid/fowfz59x"}
{"id":"id:cord19:cord19::i5fc7w6g","pathId":"/document/v1/cord19/cord19/docid/i5fc7w6g"}
{"id":"id:cord19:cord19::i5fc7w6g","pathId":"/document/v1/cord19/cord19/docid/i5fc7w6g"}
{"id":"id:cord19:cord19::5m4bxb9d","pathId":"/document/v1/cord19/cord19/docid/5m4bxb9d"}
{"id":"id:cord19:cord19::0s6lcdkf","pathId":"/document/v1/cord19/cord19/docid/0s6lcdkf"}
{"id":"id:cord19:cord19::4o8bedj8","pathId":"/document/v1/cord19/cord19/docid/4o8bedj8"}
{"id":"id:cord19:cord19::sct14hzu","pathId":"/document/v1/cord19/cord19/docid/sct14hzu"}
{"id":"id:cord19:cord

{"id":"id:cord19:cord19::xe3cvf66","pathId":"/document/v1/cord19/cord19/docid/xe3cvf66"}
{"id":"id:cord19:cord19::y1b1vf1b","pathId":"/document/v1/cord19/cord19/docid/y1b1vf1b"}
{"id":"id:cord19:cord19::bvociyg2","pathId":"/document/v1/cord19/cord19/docid/bvociyg2"}
{"id":"id:cord19:cord19::y1b1vf1b","pathId":"/document/v1/cord19/cord19/docid/y1b1vf1b"}
{"id":"id:cord19:cord19::oa8vzf02","pathId":"/document/v1/cord19/cord19/docid/oa8vzf02"}
{"id":"id:cord19:cord19::3sepefqa","pathId":"/document/v1/cord19/cord19/docid/3sepefqa"}
{"id":"id:cord19:cord19::7axcgt3j","pathId":"/document/v1/cord19/cord19/docid/7axcgt3j"}
{"id":"id:cord19:cord19::j6mfl4m8","pathId":"/document/v1/cord19/cord19/docid/j6mfl4m8"}
{"id":"id:cord19:cord19::rbt84zet","pathId":"/document/v1/cord19/cord19/docid/rbt84zet"}
{"id":"id:cord19:cord19::3hjnqf3r","pathId":"/document/v1/cord19/cord19/docid/3hjnqf3r"}
{"id":"id:cord19:cord19::e16lln3f","pathId":"/document/v1/cord19/cord19/docid/e16lln3f"}
{"id":"id:cord19:cord

{"id":"id:cord19:cord19::gxdrx6z7","pathId":"/document/v1/cord19/cord19/docid/gxdrx6z7"}
{"id":"id:cord19:cord19::idg8uz46","pathId":"/document/v1/cord19/cord19/docid/idg8uz46"}
{"id":"id:cord19:cord19::93ea9u03","pathId":"/document/v1/cord19/cord19/docid/93ea9u03"}
{"id":"id:cord19:cord19::8zqlkdez","pathId":"/document/v1/cord19/cord19/docid/8zqlkdez"}
{"id":"id:cord19:cord19::tmgmqtjq","pathId":"/document/v1/cord19/cord19/docid/tmgmqtjq"}
{"id":"id:cord19:cord19::8zqlkdez","pathId":"/document/v1/cord19/cord19/docid/8zqlkdez"}
{"id":"id:cord19:cord19::hzahejqk","pathId":"/document/v1/cord19/cord19/docid/hzahejqk"}
{"id":"id:cord19:cord19::ub1t0wwq","pathId":"/document/v1/cord19/cord19/docid/ub1t0wwq"}
{"id":"id:cord19:cord19::ub1t0wwq","pathId":"/document/v1/cord19/cord19/docid/ub1t0wwq"}
{"id":"id:cord19:cord19::7ur8hr23","pathId":"/document/v1/cord19/cord19/docid/7ur8hr23"}
{"id":"id:cord19:cord19::xa1v5t63","pathId":"/document/v1/cord19/cord19/docid/xa1v5t63"}
{"id":"id:cord19:cord

...

{"id":"id:cord19:cord19::uj4mgoqg","pathId":"/document/v1/cord19/cord19/docid/uj4mgoqg"}
{"id":"id:cord19:cord19::dbo79w8y","pathId":"/document/v1/cord19/cord19/docid/dbo79w8y"}
{"id":"id:cord19:cord19::dbo79w8y","pathId":"/document/v1/cord19/cord19/docid/dbo79w8y"}
{"id":"id:cord19:cord19::ktjevis3","pathId":"/document/v1/cord19/cord19/docid/ktjevis3"}
{"id":"id:cord19:cord19::m9xifjth","pathId":"/document/v1/cord19/cord19/docid/m9xifjth"}
{"id":"id:cord19:cord19::eduirmdd","pathId":"/document/v1/cord19/cord19/docid/eduirmdd"}
{"id":"id:cord19:cord19::l4n9hwcx","pathId":"/document/v1/cord19/cord19/docid/l4n9hwcx"}
{"id":"id:cord19:cord19::xcu80tcp","pathId":"/document/v1/cord19/cord19/docid/xcu80tcp"}
{"id":"id:cord19:cord19::rpdlk1b6","pathId":"/document/v1/cord19/cord19/docid/rpdlk1b6"}
{"id":"id:cord19:cord19::mvsw6tyy","pathId":"/document/v1/cord19/cord19/docid/mvsw6tyy"}
{"id":"id:cord19:cord19::xz6pq0v3","pathId":"/document/v1/cord19/cord19/docid/xz6pq0v3"}
{"id":"id:cord19:cord

{"id":"id:cord19:cord19::yy96yeu9","pathId":"/document/v1/cord19/cord19/docid/yy96yeu9"}
{"id":"id:cord19:cord19::ys1af2vn","pathId":"/document/v1/cord19/cord19/docid/ys1af2vn"}
{"id":"id:cord19:cord19::9xja07au","pathId":"/document/v1/cord19/cord19/docid/9xja07au"}
{"id":"id:cord19:cord19::kaip0d1i","pathId":"/document/v1/cord19/cord19/docid/kaip0d1i"}
{"id":"id:cord19:cord19::lhqvi3j0","pathId":"/document/v1/cord19/cord19/docid/lhqvi3j0"}
{"id":"id:cord19:cord19::2j6rp5ar","pathId":"/document/v1/cord19/cord19/docid/2j6rp5ar"}
{"id":"id:cord19:cord19::kxfpt7hy","pathId":"/document/v1/cord19/cord19/docid/kxfpt7hy"}
{"id":"id:cord19:cord19::9ae26r2z","pathId":"/document/v1/cord19/cord19/docid/9ae26r2z"}
{"id":"id:cord19:cord19::o7kydesb","pathId":"/document/v1/cord19/cord19/docid/o7kydesb"}
{"id":"id:cord19:cord19::by854ff6","pathId":"/document/v1/cord19/cord19/docid/by854ff6"}
{"id":"id:cord19:cord19::do478xpv","pathId":"/document/v1/cord19/cord19/docid/do478xpv"}
{"id":"id:cord19:cord

{"id":"id:cord19:cord19::de7t251s","pathId":"/document/v1/cord19/cord19/docid/de7t251s"}
{"id":"id:cord19:cord19::4f6h7agt","pathId":"/document/v1/cord19/cord19/docid/4f6h7agt"}
{"id":"id:cord19:cord19::ul43uuo3","pathId":"/document/v1/cord19/cord19/docid/ul43uuo3"}
{"id":"id:cord19:cord19::c4vv541o","pathId":"/document/v1/cord19/cord19/docid/c4vv541o"}
{"id":"id:cord19:cord19::u7ob53mr","pathId":"/document/v1/cord19/cord19/docid/u7ob53mr"}
{"id":"id:cord19:cord19::060c3u8v","pathId":"/document/v1/cord19/cord19/docid/060c3u8v"}
{"id":"id:cord19:cord19::v6n7wgi0","pathId":"/document/v1/cord19/cord19/docid/v6n7wgi0"}
{"id":"id:cord19:cord19::uw7qt4jd","pathId":"/document/v1/cord19/cord19/docid/uw7qt4jd"}
{"id":"id:cord19:cord19::n48hbt6g","pathId":"/document/v1/cord19/cord19/docid/n48hbt6g"}
{"id":"id:cord19:cord19::ptnx5ifg","pathId":"/document/v1/cord19/cord19/docid/ptnx5ifg"}
{"id":"id:cord19:cord19::716idcpz","pathId":"/document/v1/cord19/cord19/docid/716idcpz"}
{"id":"id:cord19:cord

{"id":"id:cord19:cord19::d2wau7rl","pathId":"/document/v1/cord19/cord19/docid/d2wau7rl"}
{"id":"id:cord19:cord19::ksbha7kz","pathId":"/document/v1/cord19/cord19/docid/ksbha7kz"}
{"id":"id:cord19:cord19::wecjrlex","pathId":"/document/v1/cord19/cord19/docid/wecjrlex"}
{"id":"id:cord19:cord19::uxs9tiuk","pathId":"/document/v1/cord19/cord19/docid/uxs9tiuk"}
{"id":"id:cord19:cord19::7wthyc3z","pathId":"/document/v1/cord19/cord19/docid/7wthyc3z"}
{"id":"id:cord19:cord19::oqgw6ao6","pathId":"/document/v1/cord19/cord19/docid/oqgw6ao6"}
{"id":"id:cord19:cord19::oh67pq5f","pathId":"/document/v1/cord19/cord19/docid/oh67pq5f"}
{"id":"id:cord19:cord19::dnt52isd","pathId":"/document/v1/cord19/cord19/docid/dnt52isd"}
{"id":"id:cord19:cord19::gc8770l9","pathId":"/document/v1/cord19/cord19/docid/gc8770l9"}
{"id":"id:cord19:cord19::f9nu1h3i","pathId":"/document/v1/cord19/cord19/docid/f9nu1h3i"}
{"id":"id:cord19:cord19::s0vqqgfq","pathId":"/document/v1/cord19/cord19/docid/s0vqqgfq"}
{"id":"id:cord19:cord

{"id":"id:cord19:cord19::k8xuv5xy","pathId":"/document/v1/cord19/cord19/docid/k8xuv5xy"}
{"id":"id:cord19:cord19::q192b187","pathId":"/document/v1/cord19/cord19/docid/q192b187"}
{"id":"id:cord19:cord19::3oak7rpg","pathId":"/document/v1/cord19/cord19/docid/3oak7rpg"}
{"id":"id:cord19:cord19::m8ivs34u","pathId":"/document/v1/cord19/cord19/docid/m8ivs34u"}
{"id":"id:cord19:cord19::mtdedpiw","pathId":"/document/v1/cord19/cord19/docid/mtdedpiw"}
{"id":"id:cord19:cord19::kytnsrkq","pathId":"/document/v1/cord19/cord19/docid/kytnsrkq"}
{"id":"id:cord19:cord19::wjunk1j1","pathId":"/document/v1/cord19/cord19/docid/wjunk1j1"}
{"id":"id:cord19:cord19::je9t4i6q","pathId":"/document/v1/cord19/cord19/docid/je9t4i6q"}
{"id":"id:cord19:cord19::bqhml1yo","pathId":"/document/v1/cord19/cord19/docid/bqhml1yo"}
{"id":"id:cord19:cord19::osbxx7h5","pathId":"/document/v1/cord19/cord19/docid/osbxx7h5"}
{"id":"id:cord19:cord19::xet9gz1h","pathId":"/document/v1/cord19/cord19/docid/xet9gz1h"}
{"id":"id:cord19:cord

In [5]:
from vespa.application import Vespa

app = Vespa(url = "http://localhost", port = 8080)

In [6]:
from vespa.query import Query, OR, RankProfile as Ranking

query = 'coronavirus origin'
result = app.query(
    query=query, 
    query_model=Query(
        match_phase = OR(),
        rank_profile = Ranking(name="default")),
    timeout="20s",    
)

In [7]:
result.json

{'root': {'id': 'toplevel',
  'relevance': 1.0,
  'fields': {'totalCount': 1164},
  'coverage': {'coverage': 100,
   'documents': 4593,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'children': [{'id': 'id:cord19:cord19::4dtk1kyh',
    'relevance': 0.28534213701925226,
    'source': 'c_content',
    'fields': {'sddocname': 'cord19',
     'documentid': 'id:cord19:cord19::4dtk1kyh',
     'cord_uid': '4dtk1kyh',
     'title': 'Origin of Novel Coronavirus (COVID-19): A Computational Biology Study using Artificial Intelligence',
     'title_token_ids': {'cells': [{'address': {'d0': '0'}, 'value': 4761.0},
       {'address': {'d0': '1'}, 'value': 1997.0},
       {'address': {'d0': '2'}, 'value': 3117.0},
       {'address': {'d0': '3'}, 'value': 21887.0},
       {'address': {'d0': '4'}, 'value': 23350.0},
       {'address': {'d0': '5'}, 'value': 1006.0},
       {'address': {'d0': '6'}, 'value': 2522.0},
       {'address': {'d0': '7'}, 'value': 17258.0},
       {'add

In [26]:
from vespa.query import RankProfile as Ranking

query = 'coronavirus origin'
result = app.query(
    query=query, 
    query_model=Query(
        match_phase = OR(),
        rank_profile = Ranking(name="bert_index_1")),
    timeout="20s",
    debug_request=False,
    **{"ranking.features.query(query_token_ids)": str(tokenizer(
                str(query), 
                truncation=True, 
                padding="max_length",
                max_length=64, 
                add_special_tokens=False
            )["input_ids"])}
    
)

In [33]:
[hit["relevance"] for hit in result.hits]

[-0.4861012101173401,
 -0.6376100182533264,
 -0.6376100182533264,
 -0.7449832558631897,
 -0.7505355477333069,
 -0.8059513568878174,
 -0.8117672204971313,
 -0.8244771957397461,
 -1.0382534265518188,
 -1.0917185544967651]

In [264]:
result.request_body

{'yql': 'select * from sources * where ([{"grammar": "any"}]userInput("coronavirus origin"));',
 'ranking': {'profile': 'bert', 'listFeatures': 'false'},
 'timeout': '20s',
 'tracelevel': 5,
 'ranking.features.query(query_token_ids)': '[21887, 23350, 4761, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'}

In [27]:
result.json

{'root': {'id': 'toplevel',
  'relevance': 1.0,
  'fields': {'totalCount': 1164},
  'coverage': {'coverage': 100,
   'documents': 4593,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'children': [{'id': 'id:cord19:cord19::yzp9wjuk',
    'relevance': 0.8190119862556458,
    'source': 'c_content',
    'fields': {'sddocname': 'cord19',
     'documentid': 'id:cord19:cord19::yzp9wjuk',
     'cord_uid': 'yzp9wjuk',
     'title': 'Middle East Respiratory Syndrome Coronavirus (MERS-CoV) origin and animal reservoir',
     'title_token_ids': {'cells': [{'address': {'d0': '0'}, 'value': 2690.0},
       {'address': {'d0': '1'}, 'value': 2264.0},
       {'address': {'d0': '2'}, 'value': 16464.0},
       {'address': {'d0': '3'}, 'value': 8715.0},
       {'address': {'d0': '4'}, 'value': 21887.0},
       {'address': {'d0': '5'}, 'value': 23350.0},
       {'address': {'d0': '6'}, 'value': 1006.0},
       {'address': {'d0': '7'}, 'value': 21442.0},
       {'address': {'d0': '8

### Define query models that we want to evaluate

In [12]:
from vespa.query import Query, RankProfile, OR

query_models = {
    "or_bm25": Query(
        match_phase = OR(),
        rank_profile = Ranking(name="bm25")
    ),
    "or_bm25_bert": Query(
        match_phase = OR(),
        rank_profile = Ranking(name="bert")
    ),
    "or_bm25_bert_index_1": Query(
        match_phase = OR(),
        rank_profile = Ranking(name="bert_index_1")
    )
    
}
        

In [13]:
from vespa.evaluation import MatchRatio, Recall, ReciprocalRank, NormalizedDiscountedCumulativeGain

eval_metrics = [MatchRatio(), Recall(at=10), ReciprocalRank(at=10), NormalizedDiscountedCumulativeGain(at=10)]

In [ ]:
        evaluation = []
        for query_data in labelled_data:
            evaluation_query = self.evaluate_query(
                eval_metrics=eval_metrics,
                query_model=query_model,
                query_id=query_data["query_id"],
                query=query_data["query"],
                id_field=id_field,
                relevant_docs=query_data["relevant_docs"],
                default_score=default_score,
                **kwargs
            )
            evaluation.append(evaluation_query)
        evaluation = DataFrame.from_records(evaluation)


In [49]:
for test in test_sets:
    print(test)

partially_observed
unobserved


In [59]:
query_data["relevant_docs"]

[{'id': '03dzqten', 'score': 1},
 {'id': '05djnz4p', 'score': 1},
 {'id': '066rysjh', 'score': 2},
 {'id': '06gbt9t0', 'score': 1},
 {'id': '08mjfs83', 'score': 2},
 {'id': '0phcscz8', 'score': 2},
 {'id': '0tn06al2', 'score': 2},
 {'id': '103c8wtt', 'score': 1},
 {'id': '12edypl7', 'score': 1},
 {'id': '15gnilwy', 'score': 2},
 {'id': '164yx77a', 'score': 1},
 {'id': '18h0maxi', 'score': 1},
 {'id': '1jeh7hke', 'score': 2},
 {'id': '1qc153nu', 'score': 1},
 {'id': '1s29u6tq', 'score': 2},
 {'id': '1vcvwkwn', 'score': 1},
 {'id': '1y8crjqn', 'score': 2},
 {'id': '2ros9m9p', 'score': 2},
 {'id': '380az9lp', 'score': 1},
 {'id': '38mhmxvd', 'score': 2},
 {'id': '3cr5x88g', 'score': 1},
 {'id': '3gbd9g5z', 'score': 2},
 {'id': '3iodx9dj', 'score': 1},
 {'id': '3nnlkbph', 'score': 2},
 {'id': '3tgvvq4o', 'score': 1},
 {'id': '42t0zriz', 'score': 2},
 {'id': '49q2xxkw', 'score': 2},
 {'id': '4akxfetl', 'score': 2},
 {'id': '4q72jj2h', 'score': 2},
 {'id': '53j3ly3v', 'score': 2},
 {'id': '5

In [17]:
from pandas import DataFrame

evaluations = {}
for test_set in test_sets:
    evaluations[test_set] = {}
    for query_model in query_models:
        evaluation = []
        for query_data in test_sets[test_set]:
            print(query_data["query_id"])
            evaluation_query = app.evaluate_query(
                eval_metrics=eval_metrics,
                query_model=query_models[query_model],
                query_id=query_data["query_id"],
                query=query_data["query"],
                id_field = "cord_uid",
                relevant_docs=query_data["relevant_docs"],
                hits = 10,
                timeout="100s",
                **{"ranking.features.query(query_token_ids)": str(tokenizer(
                            str(query_data["query"]), 
                            truncation=True, 
                            padding="max_length",
                            max_length=64, 
                            add_special_tokens=False
                        )["input_ids"])}            
            )
            evaluation.append(evaluation_query)
        evaluations[test_set][query_model] = DataFrame.from_records(evaluation)

1
2
4
5
6
7
8
9
11
12
14
16
17
18
19
20
21
22
23
24
25
26
28
29
30
31
32
33
35
36
37
38
39
40
42
43
44
45
47
48
50
1
2
4
5
6
7
8
9
11
12
14
16
17
18
19
20
21
22
23
24
25
26
28
29
30
31
32
33
35
36
37
38
39
40
42
43
44
45
47
48
50
1
2
4
5
6
7
8
9
11
12
14
16
17
18
19
20
21
22
23
24
25
26
28
29
30
31
32
33
35
36
37
38
39
40
42
43
44
45
47
48
50
3
10
13
15
27
34
41
46
49
3
10
13
15
27
34
41
46
49
3
10
13
15
27
34
41
46
49


In [18]:
evaluations

{'partially_observed': {'or_bm25':     query_id  match_ratio_retrieved_docs  match_ratio_docs_available  \
  0          1                        1164                        4593   
  1          2                        1709                        4593   
  2          4                        2622                        4593   
  3          5                        3606                        4593   
  4          6                        1290                        4593   
  5          7                        1892                        4593   
  6          8                        1200                        4593   
  7          9                        2554                        4593   
  8         11                        1160                        4593   
  9         12                        1182                        4593   
  10        14                        1225                        4593   
  11        16                        1628                        4593   
  12 

In [19]:
import pandas as pd

metric_values = []
for test_set in test_sets:
    for query_model in query_models:
        for metric in eval_metrics:
            metric_values.append(
                pd.DataFrame(
                    data={
                        "test_set": test_set, 
                        "query_model": query_model, 
                        "metric": metric.name, 
                        "value": evaluations[test_set][query_model][metric.name + "_value"].to_list()
                    }
                )
            )
metric_values = pd.concat(metric_values, ignore_index=True)

In [20]:
metric_values.head()

,test_set,query_model,metric,value
0,partially_observed,or_bm25,match_ratio,0.253429
1,partially_observed,or_bm25,match_ratio,0.372088
2,partially_observed,or_bm25,match_ratio,0.570869
3,partially_observed,or_bm25,match_ratio,0.785108
4,partially_observed,or_bm25,match_ratio,0.280862


In [21]:
metric_values.metric.unique()

array(['match_ratio', 'recall_10', 'reciprocal_rank_10', 'ndcg_10'],
      dtype=object)

In [75]:
import plotly.express as px


fig = px.box(metric_values[metric_values.metric == "reciprocal_rank_10"], x="query_model", y="value", title = "RR @ 10")
fig.show()

In [23]:
metric_values.groupby(['query_model', 'metric']).median()

value
query_model          metric                      
or_bm25              match_ratio         0.267472
                     ndcg_10             0.357478
                     recall_10           0.007576
                     reciprocal_rank_10  0.142857
or_bm25_bert         match_ratio         0.267472
                     ndcg_10             0.386853
                     recall_10           0.008024
                     reciprocal_rank_10  0.183333
or_bm25_bert_index_1 match_ratio         0.267472
                     ndcg_10             0.386853
                     recall_10           0.008024
                     reciprocal_rank_10  0.200000